# Importing Libraries

In [1]:
from utils import ArmHandler
from ultralytics import YOLO

import random
import time
import cv2
import math

arm = ArmHandler() # Initialize the arm handler

SDK_VERSION: 1.13.30


# Start Video Capture

In [2]:
cap = cv2.VideoCapture(1)
cap.set(3, 640)
cap.set(4, 480)
cv2.namedWindow("Webcam")
model = YOLO("yolo_models/best100.torchscript", task="detect")

# Set the pixel to world coordinate conversion

In [3]:
m1, c1 = 0.6515, 150.83251  # Get these values after coordinate calibration
m2, c2 = 0.65973, -88.76217


def pixel_to_world(pixel_x, pixel_y):
    return m1 * pixel_x + c1, m2 * pixel_y + c2

# Live Detection

In [15]:
while True:
    success, img = cap.read()
    results = model(img)

    # coordinates
    for r in results:
        boxes = r.boxes
        centeroids = []

        for box in boxes:
            confidence: float = math.ceil((box.conf[0] * 100)) / 100
            if confidence < 0.9:
                continue

            x1, y1, x2, y2 = map(int, box.xyxy[0])

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

            # Find center of the box
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            centeroids.append((center_x, center_y))

            cv2.putText(img, str(confidence), [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.circle(img, (center_x, center_y), 5, (0, 0, 255), -1)

        if centeroids:
            # px, py = sorted(centeroids, key=lambda x: x[0], reverse=True)[0] # If you want to sort from the highest (Ry)
            px, py = random.choice(centeroids)
            rx, ry = pixel_to_world(py, px)
            if arm.pickup_block_from(rx, ry):
                arm.place_block_on()
                time.sleep(20)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break


0: 640x640 21 colorful-cubes, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Out of RANGE

0: 640x640 19 colorful-cubes, 0.0ms
Speed: 2.0ms preprocess, 0.0ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 19 colorful-cubes, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 18 colorful-cubes, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 17 colorful-cubes, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 15.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 19 colorful-cubes, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 15.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 18 colorful-cubes, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 18 colorful-cubes, 0.0ms
Speed: 0.0ms p

# Release the resources

In [5]:
cap.release()
cv2.destroyAllWindows()
arm.disconnect()